In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd # parallel processing for pandas (to read large datasets) fixed memory issue
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('wordnet')

C:\Users\kedar\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
# reading data
data_comments = dd.read_csv('C:\\Users\\kedar\\OneDrive\\Desktop\\ML mini project\\DiseaseOutbreak_Dataset\\the-reddit-covid-dataset-comments.csv',on_bad_lines="skip",encoding = 'utf-8',engine='python',dtype={'created_utc': 'object', 'score': 'object'}, assume_missing=True)
data_comments = data_comments.compute()

MemoryError: 

In [3]:
data_posts = pd.read_csv('C:\\Users\\kedar\\OneDrive\\Desktop\\ML mini project\\DiseaseOutbreak_Dataset\\the-reddit-covid-dataset-posts.csv',encoding = 'utf-8')

In [4]:
data_comments.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,hi1vsag,2riyy,nova,False,1635206399,https://old.reddit.com/r/nova/comments/qfs53d/...,When you scheduled your booster with CVS does ...,0.0000,2
1,comment,hi1vs7i,2qhov,vancouver,False,1635206397,https://old.reddit.com/r/vancouver/comments/qf...,Didn't stop prices there though. New Zealand a...,0.1887,32
2,comment,hi1vs5n,2qwzb,pregnant,False,1635206397,https://old.reddit.com/r/pregnant/comments/qfs...,I’m just waiting until after pregnancy to get ...,0.6720,1
3,comment,hi1vs5v,2qixm,startrek,False,1635206397,https://old.reddit.com/r/startrek/comments/qft...,*The first duty of every Starfleet officer is ...,0.9562,1
4,comment,hi1vs0l,2qsf3,ontario,False,1635206395,https://old.reddit.com/r/ontario/comments/qfkj...,Compare BC to Ontario for COVID. It's even mor...,0.0000,-2


In [9]:
# taking care of missing values
# Count NaN values in each column
non_null_comments = data_comments.count()
nan_comments = non_null_comments.sum().compute()
# nan_comments = compute(nan_comments)[0]
nan_posts = data_posts.isna().sum()
print('Nan values in column of Comments dataset: ',nan_comments)
print('Nan values in each column of Posts dataset: ',nan_posts)
# dropping 'url' and 'selftext' columns in psosts dataset
columns_to_drop = ['url','selftext']
data_posts = data_posts.drop(columns=columns_to_drop)

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-------------+---------+----------+
| Column      | Found   | Expected |
+-------------+---------+----------+
| created_utc | float64 | int64    |
| score       | float64 | int64    |
+-------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'created_utc': 'float64',
       'score': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [ ]:
# preprocessing data 
comments_attributes  = data_comments.columns
posts_attributes = data_posts.columns
print("Columns in Comments Dataset: ",comments_attributes)
print("Columns in Posts Dataset: ",posts_attributes)

# cleaning up text using NLP methods
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters, numbers, and symbols
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove stopwords - Articles ,Prepositions ,Conjunctions,Pronouns,Common verbs
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    # Lemmatization - converting words to their base form such as running to run, better to good etc.
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# applying cleaning methods only to reuired columns in the dataset
data_comments['body'] = data_comments['body'].apply(clean_text)
data_posts['selftext'] = data_posts['selftext'].apply(clean_text)
data_posts['title'] = data_posts['title'].apply(clean_text)

#converting utc to datetime method
data_comments['created_utc'] = pd.to_datetime(data_comments['created_utc'])
data_posts['created_utc'] = pd.to_datetime(data_posts['created_utc'])